# 04 — Agentic Playground

This notebook is the **agent layer** for the Dimensional UI Hypercube.

Instead of just hard-coding themes or geometry, I:

- Load motion patterns (or fall back to a dummy pattern)
- Build a compact **summary** of user behavior
- Use simple **"agent" functions** to decide:
  - which theme to activate (Desert / Water / Neon)
  - which geometry mode to prefer (Cube / Sphere / Water)
  - which depth layout to use (Flat / Layered / Deep)

Right now, these agents are Python functions.

Later, the same summary can be sent to:

- Google AI Studio
- an MCP server
- any LLM-based agent

to make smarter, context-aware UI decisions.


In [1]:
import json
from pathlib import Path

import pandas as pd

# Try to load patterns from Kaggle dataset
DATA_DIR = Path("/kaggle/input/dimensional-ui-patterns")  # change slug if needed

print("DATA_DIR:", DATA_DIR)
print("DATA_DIR exists:", DATA_DIR.exists())

pattern_files = []
if DATA_DIR.exists():
    pattern_files = list(DATA_DIR.rglob("*.json"))

print("JSON files found:", len(pattern_files))
for p in pattern_files:
    print(" -", p)


DATA_DIR: /kaggle/input/dimensional-ui-patterns
DATA_DIR exists: False
JSON files found: 0


In [2]:
rows = []

# 1) Try to load real JSON files first
if len(pattern_files) > 0:
    print("🔎 Loading real pattern files...")
    for file in pattern_files:
        try:
            with open(file, "r", encoding="utf-8") as f:
                raw = json.load(f)

            pid = raw.get("Id", file.name)
            user_id = raw.get("UserId", "unknown")
            label = raw.get("Label", file.name)

            samples = raw.get("Samples", []) or raw.get("samples", [])

            for s in samples:
                rows.append(
                    {
                        "pattern_id": pid,
                        "user_id": user_id,
                        "label": label,
                        "x": s.get("X", s.get("x")),
                        "y": s.get("Y", s.get("y")),
                        "timestamp_ms": s.get(
                            "TimestampMs", s.get("timestamp_ms")
                        ),
                        "gesture_type": s.get(
                            "GestureType", s.get("gestureType")
                        ),
                        "shape_mode": s.get(
                            "ShapeMode", s.get("shapeMode")
                        ),
                        "theme": s.get("Theme", s.get("theme")),
                    }
                )
        except Exception as e:
            print("⚠ Error parsing", file, ":", e)

# 2) If still nothing, build a dummy dataset
if len(rows) == 0:
    print("⚠ No real data found. Using dummy dataset instead.")
    for i in range(50):
        rows.append(
            {
                "pattern_id": "dummy_drag_01",
                "user_id": "demo_user",
                "label": "Dummy desert drag",
                "x": 100 + i * 4,
                "y": 200 + (i * (-2 if i % 2 == 0 else 2)),
                "timestamp_ms": 1700000000000 + i * 16,
                "gesture_type": 2,  # Drag
                "shape_mode": 0,    # Cube
                "theme": 0,         # Desert
            }
        )

df = pd.DataFrame(rows)
print("Rows loaded:", len(df))
print("Columns:", list(df.columns))
df.head()


⚠ No real data found. Using dummy dataset instead.
Rows loaded: 50
Columns: ['pattern_id', 'user_id', 'label', 'x', 'y', 'timestamp_ms', 'gesture_type', 'shape_mode', 'theme']


,pattern_id,user_id,label,x,y,timestamp_ms,gesture_type,shape_mode,theme
0,dummy_drag_01,demo_user,Dummy desert drag,100,200,1700000000000,2,0,0
1,dummy_drag_01,demo_user,Dummy desert drag,104,202,1700000000016,2,0,0
2,dummy_drag_01,demo_user,Dummy desert drag,108,196,1700000000032,2,0,0
3,dummy_drag_01,demo_user,Dummy desert drag,112,206,1700000000048,2,0,0
4,dummy_drag_01,demo_user,Dummy desert drag,116,192,1700000000064,2,0,0


In [3]:
# Maps for readability
GESTURE_MAP = {
    0: "Click",
    1: "Hold",
    2: "Drag",
    3: "Scroll",
    4: "RightClick",
}

SHAPE_MAP = {
    0: "Cube",
    1: "Sphere",
    2: "Water",
}

THEME_MAP = {
    0: "Desert",
    1: "Water",
    2: "Neon",
}

df["gesture_name"] = df["gesture_type"].map(GESTURE_MAP)
df["shape_name"]   = df["shape_mode"].map(SHAPE_MAP)
df["theme_name"]   = df["theme"].map(THEME_MAP)

summary = {
    "num_patterns": int(df["pattern_id"].nunique()),
    "num_samples": int(len(df)),
    "by_theme": df["theme"].value_counts().to_dict(),
    "by_shape_mode": df["shape_mode"].value_counts().to_dict(),
    "by_gesture_type": df["gesture_type"].value_counts().to_dict(),
}

summary


{'num_patterns': 1,
 'num_samples': 50,
 'by_theme': {0: 50},
 'by_shape_mode': {0: 50},
 'by_gesture_type': {2: 50}}

In [4]:
def agent_recommend_theme(summary_dict):
    """
    Decide which theme to activate.
    For now:
      - pick the most-used theme id
      - map to name (Desert / Water / Neon)
    """
    theme_counts = summary_dict.get("by_theme", {})
    if not theme_counts:
        return "Desert"

    THEME_MAP = {0: "Desert", 1: "Water", 2: "Neon"}
    best_theme_id = max(theme_counts, key=theme_counts.get)
    return THEME_MAP.get(best_theme_id, "Desert")


def agent_recommend_shape_mode(summary_dict):
    """
    Decide which geometry mode to prefer (Cube / Sphere / Water).
    For now:
      - pick the most-used shape_mode id
    """
    shape_counts = summary_dict.get("by_shape_mode", {})
    if not shape_counts:
        return "Cube"

    SHAPE_MAP = {0: "Cube", 1: "Sphere", 2: "Water"}
    best_shape_id = max(shape_counts, key=shape_counts.get)
    return SHAPE_MAP.get(best_shape_id, "Cube")


def agent_recommend_depth_layout(summary_dict):
    """
    Decide how deep the UI should feel:
      - 'Flat'       → low pattern complexity / few patterns
      - 'Layered'    → medium activity
      - 'Deep'       → many patterns / samples
    """
    num_patterns = summary_dict.get("num_patterns", 0)
    num_samples  = summary_dict.get("num_samples", 0)

    if num_patterns <= 1 and num_samples < 80:
        return "Flat"
    elif num_patterns <= 3 and num_samples < 300:
        return "Layered"
    else:
        return "Deep"


In [5]:
recommended_theme  = agent_recommend_theme(summary)
recommended_shape  = agent_recommend_shape_mode(summary)
recommended_layout = agent_recommend_depth_layout(summary)

print("Agent recommendation:")
print("- Theme:       ", recommended_theme)
print("- Shape mode:  ", recommended_shape)
print("- Depth layout:", recommended_layout)


Agent recommendation:
- Theme:        Desert
- Shape mode:   Cube
- Depth layout: Flat


In [6]:
agent_context = {
    "ui_state": {
        "current_theme": recommended_theme,
        "current_shape_mode": recommended_shape,
        "current_layout_depth": recommended_layout,
    },
    "usage_summary": summary,
    "notes": {
        "description": (
            "This context represents aggregated motion patterns for the "
            "Dimensional UI Hypercube. An agent can update theme, shape mode, "
            "or layout depth based on this summary."
        )
    },
}

agent_context


{'ui_state': {'current_theme': 'Desert',
  'current_shape_mode': 'Cube',
  'current_layout_depth': 'Flat'},
 'usage_summary': {'num_patterns': 1,
  'num_samples': 50,
  'by_theme': {0: 50},
  'by_shape_mode': {0: 50},
  'by_gesture_type': {2: 50}},
 'notes': {'description': 'This context represents aggregated motion patterns for the Dimensional UI Hypercube. An agent can update theme, shape mode, or layout depth based on this summary.'}}

In [7]:
import json

context_json = json.dumps(agent_context, indent=2)

pseudo_prompt = f"""
You are an interface orchestration agent for a Dimensional UI Hypercube.

Here is the current context (JSON):

{context_json}

Based on this:
- Decide whether to keep or change the theme (Desert / Water / Neon)
- Decide whether to keep or change the shape mode (Cube / Sphere / Water)
- Decide whether to keep Flat / Layered / Deep layout
- Return your decision in JSON with keys: 'theme', 'shape_mode', 'layout_depth', 'reasoning'.
"""

print(pseudo_prompt[:1200], "...\n\n[truncated]")



You are an interface orchestration agent for a Dimensional UI Hypercube.

Here is the current context (JSON):

{
  "ui_state": {
    "current_theme": "Desert",
    "current_shape_mode": "Cube",
    "current_layout_depth": "Flat"
  },
  "usage_summary": {
    "num_patterns": 1,
    "num_samples": 50,
    "by_theme": {
      "0": 50
    },
    "by_shape_mode": {
      "0": 50
    },
    "by_gesture_type": {
      "2": 50
    }
  },
  "notes": {
    "description": "This context represents aggregated motion patterns for the Dimensional UI Hypercube. An agent can update theme, shape mode, or layout depth based on this summary."
  }
}

Based on this:
- Decide whether to keep or change the theme (Desert / Water / Neon)
- Decide whether to keep or change the shape mode (Cube / Sphere / Water)
- Decide whether to keep Flat / Layered / Deep layout
- Return your decision in JSON with keys: 'theme', 'shape_mode', 'layout_depth', 'reasoning'.
 ...

[truncated]


## Summary

In this Agentic Playground, I:

- Loaded motion pattern data (or a dummy pattern fallback)
- Built a compact **usage summary**:
  - number of patterns / samples
  - distribution by theme / shape mode / gesture type
- Created agent stubs to decide:
  - **Theme** (Desert / Water / Neon)
  - **Shape mode** (Cube / Sphere / Water)
  - **Layout depth** (Flat / Layered / Deep)
- Bundled everything into a JSON-like `agent_context` that can be
  handed to any LLM / AI agent (Google AI Studio, MCP, etc.)

Later phases:

- Replace the Python agent stubs with real model calls
- Let the agent continuously adapt the hypercube UI:
  - switch themes based on user fatigue / intensity
  - change geometry mode for different work modes
  - deepen or flatten the layout based on activity
